#### Grounding with Gemini on Vertex Search Datastore

[Follow These Steps](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#unstructured-data_1) to leverage Vertex Agent Builder to create a Search App with unstructured data store on Google Cloud Console. 

Once the search app is created, you can leverage it's Vertex Search Datastore as a [Grounding Source](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/grounding) with Gemini Models directly to ground responses in your data.

In [ ]:
!pip install --upgrade google-cloud-aiplatform
# !gcloud auth application-default login

In [ ]:
import vertexai

from vertexai.preview.generative_models import grounding
from vertexai.generative_models import GenerativeModel, GenerationConfig, Tool

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID"
VERTEX_SEARCH_DATASTORE= "YOUR_DATASTORE_PATH" # data_store_path = "projects/{project_id}/locations/{location}/collections/default_collection/dataStores/{data_store_id}"


In [ ]:
prompt_system_instruction = """
<PERSONA_AND_GOAL>
  You are a helpful assistant knowledgeable about Alphabet's quarterly earning reports.
  Help user with their queries related with Alphabet by following given <INSTRUCTIONS> and <CONTEXT>
  only respond with information available in Grounding Knowledge store.
</PERSONA_AND_GOAL>

<INSTRUCTIONS>
- Always refer to the tool and Ground your answers in it
- Understand the retrieved snippet by the tool and only use that information to help users
- For supporting references, you can provide the Grounding tool snippets verbatim, and any other info like page number
- For Information not available in the tool, mention you dont have access to the information.
- Output "answer" should be I dont know when the user question is irrelevant or outside the <CONTEXT>
- Leave "reference_snippet" as null if you are not sure about the page and text snippet
<INSTRUCTIONS>

<CONTEXT>
  Grounding tool finds most relevant snippets from the Alphabet earning reports data store.
  Use the information provided by the tool as your knowledge base.
</CONTEXT>

<CONSTRAINTS>
- ONLY use information available from the Grounding tool
</CONSTRAINTS>

<OUTPUT_FORMAT>
- Response should ALWAYS be in following JSON Output with answer and reference_snippet as keys, e.g. {"answer": , "reference_snippet": }
</OUTPUT_FORMAT>
"""

In [ ]:
tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=VERTEX_SEARCH_DATASTORE))
)

In [ ]:
vertexai.init(project=PROJECT_ID, location="us-central1")

model = GenerativeModel(
    model_name="gemini-1.5-pro-001",
    tools=[tool],
    system_instruction=[prompt_system_instruction]
)

In [ ]:
prompt = "When does Alphabet plan to get to net zero?"
response = model.generate_content(
    prompt,
    generation_config=GenerationConfig(
        temperature=0.0
    ),
)

In [ ]:
response.candidates[0].content.parts[0].text

'```json\n{\n "answer": "Alphabet has set a goal to achieve net zero emissions across its operations and value chain by 2030.",\n "reference_snippet": "Our goal to operate our data\\ncenter and office campuses on 24/7 carbon\\nfree energy by 2030 remains one of our greatest\\nambitions, and we have made significant progress –\\nthis is part of our broader goal to achieve net zero\\nemissions across our operations and value chain,\\nincluding consumer hardware products, by 2030."\n}\n```'

In [ ]:
response.candidates[0].grounding_metadata

grounding_attributions {
  segment {
    end_index: 477
  }
  confidence_score: 0.9883381724357605
  retrieved_context {
    uri: "gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/2022_alphabet_annual_report.pdf"
    title: "2022_alphabet_annual_report"
  }
}
retrieval_queries: "When does Alphabet plan to get to net zero?"